In [ ]:
!pip install oci python-docx PyPDF2 openai langgraph langchain langchain-openai




In [ ]:
%pip install oci


In [14]:
# STEP 2: Read files from OCI Object Storage into Python

from io import BytesIO
from docx import Document   # for reading .docx files
from PyPDF2 import PdfReader  # for reading .pdf files
import oci


In [15]:
config_path = r"A:\AI Projects\oci\config.ini"  # raw string avoids issues with backslashes
config = oci.config.from_file(file_location=config_path, profile_name="DEFAULT")

In [16]:
# Create Object Storage client
object_storage_client = oci.ject_storage.ObjectStorageClient(config)ob


In [17]:
# Get namespace
namespace = object_storage_client.get_namespace().data

In [18]:
print("✅ Connected to OCI successfully!")
print("Namespace:", namespace)

✅ Connected to OCI successfully!
Namespace: axrmgaiopsw4


In [19]:
bucket_name = "policy-docs-bucket"
# List all objects in the bucket
response = object_storage_client.list_objects(namespace, bucket_name)
print("Files in bucket:")
for obj in response.data.objects:
    print(obj.name)

Files in bucket:
Prokngb.docx


In [20]:
# Read documents from OCI 
def read_docx_from_oci(object_name):
    response = object_storage_client.get_object(namespace, bucket_name, object_name)
    file_stream = BytesIO(response.data.content)
    doc = Document(file_stream)
    return "\n".join([p.text for p in doc.paragraphs])

In [21]:
# Load all documents into a Python list
docs = []

for obj in response.data.objects:
    if obj.name.endswith(".docx"):
        text = read_docx_from_oci(obj.name)
        docs.append({"filename": obj.name, "text": text})
    elif obj.name.endswith(".pdf"):
        text = read_pdf_from_oci(obj.name)
        docs.append({"filename": obj.name, "text": text})

print(f"✅ Loaded {len(docs)} documents from Object Storage")


✅ Loaded 1 documents from Object Storage


In [ ]:
from openai import OpenAI

## Genrate Embeddings using OpenAI
client  = OpenAI(api_key="your_api_key")

In [25]:
def embed_text(text):
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=text
    )
    return response.data[0].embedding

# Example: generate embeddings for all documents
for doc in docs:
    doc['embedding'] = embed_text(doc['text'])

In [27]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langgraph.graph import StateGraph, END

from langchain.schema import HumanMessage

In [ ]:
# Initialize LLM + Embeddings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key="YOUR_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key="YOUR_API_KEY")

In [ ]:
!pip install faiss-cpu
# Build FAISS vector store from docs
texts = [doc["text"] for doc in docs]
metadatas = [{"source": doc["filename"]} for doc in docs]

vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [33]:
from typing import TypedDict, List

class AgentState(TypedDict):
    question: str
    context: str
    answer: str



In [34]:
# %%
# Node 1: Retrieve relevant docs
def retrieve_node(state: AgentState):
    docs = retriever.get_relevant_documents(state["question"])
    context = "\n\n".join([d.page_content for d in docs])
    return {"context": context}

In [35]:
# Node 2: Generate answer
def answer_node(state: AgentState):
    messages = [
        HumanMessage(content=f"Answer the question based on the context.\n\nContext:\n{state['context']}\n\nQuestion: {state['question']}")
    ]
    response = llm(messages)
    return {"answer": response.content}


In [36]:
# %%
# Build LangGraph workflow
graph = StateGraph(AgentState)

graph.add_node("retrieve", retrieve_node)
graph.add_node("answer", answer_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "answer")
graph.add_edge("answer", END)

agent = graph.compile()

In [37]:
# Example query
query = "Give me basic about Oracle R12 Procurement process?"
state = {"question": query, "context": "", "answer": ""}

In [38]:
result = agent.invoke(state)
print("Q:", query)
print("A:", result["answer"])

C:\Users\sarve\AppData\Local\Temp\ipykernel_35872\2454362818.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(state["question"])
C:\Users\sarve\AppData\Local\Temp\ipykernel_35872\2964837565.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


Q: Give me basic about Oracle R12 Procurement process?
A: The Oracle R12 Procurement process is designed to streamline and automate the procure-to-pay cycle, enabling organizations to efficiently manage their purchasing activities. Here are the basic components of the process:

1. **Requisitioning**: Employees create purchase requisitions using the iProcurement module, which provides a user-friendly interface for selecting items and services.

2. **Approval Workflows**: Requisitions are routed through automated approval workflows based on predefined purchasing policies, approval hierarchies, and spending limits.

3. **Sourcing**: The Oracle Sourcing module facilitates competitive bidding and supplier negotiations to secure the best terms and prices for goods and services.

4. **Purchasing**: Once requisitions are approved, Oracle Purchasing handles the core buying activities, generating purchase orders that are sent to suppliers.

5. **Supplier Management**: The Supplier Lifecycle Mana